In [1]:
!pip install -e .

Obtaining file:///home/user/FusionTransformer
  Attempting uninstall: FusionTransformer
    Found existing installation: FusionTransformer 0.0.1
    Uninstalling FusionTransformer-0.0.1:
      Successfully uninstalled FusionTransformer-0.0.1
  Running setup.py develop for FusionTransformer


In [2]:
import numpy as np
import logging
import time

import torch
import torch.nn.functional as F

In [3]:
from tqdm import tqdm

In [4]:
from FusionTransformer.common.config import purge_cfg
from FusionTransformer.config.FusionTransformerConfig import cfg

In [5]:
from FusionTransformer.models.build import build_model
from FusionTransformer.data.build import build_dataloader

In [6]:
from FusionTransformer.common.utils.checkpoint import CheckpointerV2

In [7]:
from FusionTransformer.common.utils.torch_util import set_random_seed

In [8]:
from FusionTransformer.data.utils.evaluate import Evaluator

In [9]:
from FusionTransformer.common.utils.metric_logger import MetricLogger

In [10]:
def map_sparse_to_org(x, inverse_map):
    return x[inverse_map]

In [11]:
def spread_predictions(left_idx, right_idx, curr_seg_label, curr_points_idx, curr_inverse_map, pred_label_voxel_2d, pred_label_voxel_3d, pred_label_voxel_ensemble, map_inverse_label):
    pred_label_2d = pred_label_voxel_2d[left_idx:right_idx]
    pred_label_2d = map_sparse_to_org(pred_label_2d, curr_inverse_map)

    pred_label_3d = pred_label_voxel_3d[left_idx:right_idx] #if model_3d else None
    pred_label_3d = map_sparse_to_org(pred_label_3d, curr_inverse_map)

    pred_label_ensemble = pred_label_voxel_ensemble[left_idx:right_idx]# if model_3d else None
    pred_label_ensemble = map_sparse_to_org(pred_label_ensemble, curr_inverse_map)

    if map_inverse_label is not None:
        curr_seg_label = map_inverse_label(curr_seg_label)
        pred_label_2d =  map_inverse_label(pred_label_2d)
        pred_label_3d =  map_inverse_label(pred_label_3d)
        pred_label_ensemble = map_inverse_label(pred_label_ensemble)
            
    return pred_label_2d, pred_label_3d, pred_label_ensemble, curr_seg_label

In [12]:
def validate(cfg,
             model,
             dataloader,
#              val_metric_logger,
             class_weights
             ):
    print('FusionTransformer.validate')
    print('Validation')

    # evaluator
    class_names = dataloader.dataset.class_names
    class_labels = dataloader.dataset.class_labels
    evaluator_2d = Evaluator(class_names, labels=class_labels)
    evaluator_3d = Evaluator(class_names, labels=class_labels) #if model_3d else None
    evaluator_ensemble = Evaluator(class_names, labels=class_labels) # if model_3d else None


    end = time.time()
    with torch.no_grad():
        for iteration, data_batch in tqdm(enumerate(dataloader), total=len(dataloader)):
            data_time = time.time() - end
            # copy data from cpu to gpu
            if 'SCN' in cfg.DATASET.TYPE:
                data_batch["img"] = data_batch["img"].cuda()
                data_batch["lidar"] = data_batch["lidar"].cuda()
                # data_batch['x'][1] = data_batch['x'][1].cuda()
                data_batch['seg_label'] = data_batch['seg_label'].cuda()
                # data_batch['img'] = data_batch['img'].cuda()
            else:
                raise NotImplementedError

            # predict
#             import ipdb; ipdb.set_trace()
            preds = model(data_batch)

            # preds_2d = model_2d(data_batch)
            # preds_3d = model_3d(data_batch) if model_3d else None

            pred_label_voxel_2d = preds['img_seg_logit'].argmax(1).cpu().numpy()
            pred_label_voxel_3d = preds['lidar_seg_logit'].argmax(1).cpu().numpy() 

            # softmax average (ensembling)
            probs_2d = F.softmax(preds['img_seg_logit'], dim=1)
            probs_3d = F.softmax(preds['lidar_seg_logit'], dim=1) 
            pred_label_voxel_ensemble = (probs_2d + probs_3d).argmax(1).cpu().numpy() 


            # get original point cloud from before voxelization
            seg_label = data_batch['orig_seg_label']
            points_idx = data_batch['sparse_orig_points_idx']
            inverse_map = data_batch["inverse_map"]
            
            left_idx = 0
            for batch_ind in range(len(seg_label)):
                
                curr_points_idx = points_idx[batch_ind]
                assert np.all(curr_points_idx)
                curr_inverse_map = inverse_map[batch_ind]
                curr_seg_label = seg_label[batch_ind]
                right_idx = left_idx + curr_points_idx.sum()
                
                pred_label_2d, pred_label_3d, pred_label_ensemble, curr_seg_label = \
                spread_predictions(left_idx, right_idx, 
                                   curr_seg_label, curr_points_idx, curr_inverse_map, 
                                   pred_label_voxel_2d, pred_label_voxel_3d, pred_label_voxel_ensemble,
                                    dataloader.dataset.map_inverse_label
                                  )

                # evaluate
#                 import ipdb; ipdb.set_trace()
#                 print("prediction 2d: ", np.unique(pred_label_2d))
#                 print("prediction 3d: ", np.unique(pred_label_3d))

#                 print("label:", np.unique(curr_seg_label))
                
                evaluator_2d.update(pred_label_2d, curr_seg_label)
                # if model_3d:

                evaluator_3d.update(pred_label_3d, curr_seg_label)
                evaluator_ensemble.update(pred_label_ensemble, curr_seg_label)

                left_idx = right_idx

            seg_loss_2d = F.cross_entropy(preds['img_seg_logit'], data_batch['seg_label'], weight=class_weights)
            seg_loss_3d = F.cross_entropy(preds['lidar_seg_logit'], data_batch['seg_label'], weight=class_weights) # if model_3d else None
#             val_metric_logger.update(seg_loss_2d=seg_loss_2d)
#             if seg_loss_3d is not None:
#                 val_metric_logger.update(seg_loss_3d=seg_loss_3d)

            batch_time = time.time() - end
#             val_metric_logger.update(time=batch_time, data=data_time)
            end = time.time()
    eval_list = [('2D', evaluator_2d),('3D', evaluator_3d), ('2D+3D', evaluator_ensemble)]
#     eval_list = [('3D', evaluator_3d)]

    # if model_3d:
    eval_list.extend([])
    for modality, evaluator in eval_list:
        print('{} overall accuracy={:.2f}%'.format(modality, 100.0 * evaluator.overall_acc))
        print('{} overall IOU={:.2f}'.format(modality, 100.0 * evaluator.overall_iou))
        print('{} class-wise segmentation accuracy and IoU.\n{}'.format(modality, evaluator.print_table()))


In [13]:
config_file = "/home/user/FusionTransformer/configs/semantic_kitti/middlefusion.yaml"

In [14]:
# config_file = "/home/user/FusionTransformer/configs/semantic_kitti/earlyfusion.yaml"

In [15]:
# pretrained_model_path = "/home/user/logs_2/semantic_kitti/middlefusion/model000094.pth"
pretrained_model_path = "/home/user/logs/FusionTransformer/semantic_kitti/middlefusion/model000076.pth"

In [16]:
# pretrained_model_path = "/home/user/logs/FusionTransformer/semantic_kitti/earlyfusion/model000072.pth"

In [17]:
cfg.merge_from_file(config_file)
purge_cfg(cfg)
cfg.freeze()

In [18]:
logger = logging.getLogger('FusionTransformer.test')

model = build_model(cfg)[0]

model = model.cuda()

In [19]:
# build checkpointer
checkpointer = CheckpointerV2(model, save_dir="", logger=logger)


# load last checkpoint
checkpointer.load(pretrained_model_path, resume=False)

# build dataset
test_dataloader = build_dataloader(cfg, mode='test')

Initialize SemanticKITTI dataloader
Load ('test',)


In [20]:
# ---------------------------------------------------------------------------- #
# Test
# ---------------------------------------------------------------------------- #

set_random_seed(cfg.RNG_SEED)
# test_metric_logger = MetricLogger(delimiter='  ')
model.eval()

if cfg.TRAIN.CLASS_WEIGHTS:
    class_weights = torch.tensor(cfg.TRAIN.CLASS_WEIGHTS).cuda()
else:
    class_weights = None

In [ ]:
validate(cfg=cfg,
             model=model,
             dataloader=test_dataloader,
             class_weights=class_weights
             )

FusionTransformer.validate
Validation


  0%|          | 0/408 [00:00<?, ?it/s]

input shape torch.Size([10, 96, 370, 1226]) unique  tensor([0.]) idx  0
input shape torch.Size([10, 96, 370, 1226]) unique  tensor([-1.8336, -1.8274, -1.8149,  ...,  2.0035,  2.0041,  2.0058]) idx  1


  0%|          | 1/408 [00:08<56:49,  8.38s/it]

input shape torch.Size([10, 96, 370, 1226]) unique  tensor([0.]) idx  0
input shape torch.Size([10, 96, 370, 1226]) unique  tensor([-1.8318, -1.8149, -1.8081,  ...,  2.0025,  2.0040,  2.0041]) idx  1


...

 10%|█         | 41/408 [04:19<40:49,  6.67s/it]

input shape torch.Size([10, 96, 370, 1226]) unique  tensor([0.]) idx  0
input shape torch.Size([10, 96, 370, 1226]) unique  tensor([-1.8318, -1.8274, -1.8149,  ...,  2.0040,  2.0041,  2.0042]) idx  1


 10%|█         | 42/408 [04:26<40:22,  6.62s/it]

input shape torch.Size([10, 96, 370, 1226]) unique  tensor([0.]) idx  0
input shape torch.Size([10, 96, 370, 1226]) unique  tensor([-1.8318, -1.8274, -1.8149,  ...,  2.0035,  2.0055,  2.0069]) idx  1


 11%|█         | 43/408 [04:32<39:17,  6.46s/it]